In [1]:
import meshutil
import stl.mesh
import numpy
import trimesh

In [2]:
b0 = numpy.array([
    [0, 0, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 0],
], dtype=numpy.float64) - [0.5, 0.5, 0]
xf0_to_1 = meshutil.Transform().translate(0,0,1)
b1 = xf0_to_1.apply_to(b0)

In [7]:
mesh_fname = "cube_test.stl"

mesh = meshutil.join_boundary_simple(b0, b1)
mesh = mesh.concat(meshutil.close_boundary_simple(b0))
for i in range(4):
    # Opening boundary:
    b = b1
    xf = meshutil.Transform() \
        .translate(0,0,-1) \
        .scale(0.5) \
        .translate(0.25,0.25,1) \
        .rotate([0,0,1], i*numpy.pi/2)
    for layer in range(30):
        b_sub0 = xf.apply_to(b)
        incr = meshutil.Transform() \
            .scale(0.9) \
            .rotate([-1,0,1], 0.3) \
            .translate(0,0,0.8)
        b_sub1 = incr.compose(xf).apply_to(b)
        m = meshutil.join_boundary_simple(b_sub0, b_sub1)
        mesh = mesh.concat(m)
        xf = incr.compose(xf)
    # Close final boundary:
    mesh = mesh.concat(meshutil.close_boundary_simple(b_sub1[::-1,:]))
    # ::-1 is to reverse the boundary's order to fix winding order.
    # Not sure of the "right" way to fix winding order here.
    # The boundary vertices go in an identical order... it's just
    # that clockwise/counter-clockwise flip.

# I keep confusing the 'incremental' transform with the
# transform to get b_open in the first place
    
# rotate([-1,0,1], ang).rotate([0,0,1], i*numpy.pi/2)

# I don't need to subdivide *geometry*.
# I need to subdivide *space* and then put geometry in it.

# Work around some annoying-ass trimesh/threejs bug:
wtf = meshutil.Transform().scale(0.1).translate(5,0,0)
center = meshutil.Transform().translate(-0.5, -0.5, -0.5)
base = meshutil.cube(open_xz=False).transform(center)
base = base.transform(wtf)
# to enable:
#mesh = mesh.concat(base)
mesh.to_stl_mesh().save(mesh_fname)

In [8]:
# TODO: Just use the mesh data directly (no sense in saving & re-loading)
m = trimesh.load_mesh(mesh_fname)
#m.show()
scene = trimesh.Scene([m])
scene.show()

face_normals didn't match triangles, ignoring!
